<a href="https://colab.research.google.com/github/geeksid1800/goSwift_geeksid/blob/main/goSwift_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

!pip install fuzzywuzzy
!pip install python-Levenshtein

import fuzzywuzzy
from fuzzywuzzy import process
import chardet

#utf-8 unable to recognize 'Home decor'
input_data = pd.read_csv('sample_data.csv',encoding='latin1') 

In [2]:
print(input_data.shape[0])

# convert to lower case
input_data['items.category'] = input_data['items.category'].str.lower()
# remove trailing white spaces
input_data['items.category'] = input_data['items.category'].str.strip()
#cleaning up the video card name format
input_data['items.category'] = input_data['items.category'].str.split('|').str[0]
input_data['items.category'] = input_data['items.category'].str.split('\:').str[-1].str.strip()

input_data.drop_duplicates(inplace=True)
print(input_data.shape[0])
cats = input_data['items.category'].unique()
cats.sort()
cats

28001
114


array(['accessories', 'apparel', 'apparels', 'apparels test',
       'automotive', 'ayurvedic', 'ayurvedic medicine', 'bath essentials',
       'bath scrub', 'bathing essentials', 'bathroom suppliess',
       'battery (any kind)', 'beauty', 'beauty & persomal care',
       'beauty & personal', 'beauty & personal care', 'beauty-beauty',
       'body care', 'books & stationery', 'bracelet', 'can', 'chain',
       'chain pendant', 'chandbali', 'chandelier', 'chyawanprash range',
       'clothes', 'cosmetics', 'crockery', 'danglers', 'diaper',
       'diapers', 'diy tool', 'diy tools', 'earring', 'earring set',
       'electronic', 'embroidery supplies', 'essential epquipment',
       'essentialgoods', 'fake jewellery', 'fake jewelry',
       'faucet extention', 'food', 'food (liquid)', 'food (solid)',
       'food ingredients', 'food product', 'food products', 'gardening',
       'geo grow bag', 'glassware', 'health & personal care',
       'helth & personal care', 'home & kitchen', 'home

In [ ]:
matches = fuzzywuzzy.process.extract("video card (1a01yf907)", cats, limit=10, scorer=fuzzywuzzy.fuzz.partial_ratio)
matches

In [4]:
import networkx 
from networkx.algorithms.components.connected import connected_components


def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

In [5]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(DF_list,df, column, string_to_match, min_ratio = 70):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.partial_ratio)

    # only get matches with a ratio > min_ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]
    #current_set = pd.DataFrame(close_matches, columns = ["cluster"])
    #print(current_set,end='\n\n')
    DF_list.append(close_matches)
    # get the rows of all the close matches in our dataframe
    #rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    #df.loc[rows_with_matches, column] = string_to_match

In [15]:
#create a list of the unique categories, we can merge common ones later
cat_list = list()

for current_cat in input_data['items.category']:
  replace_matches_in_column(DF_list = cat_list, df=input_data, column='items.category', string_to_match=current_cat)

#for cat in cat_list:
 # print(cat)

#Merging any categories that have common elements. This is because for each item we create a new cluster
#So there are a lot of redundant clusters
#But Using this makes a lot of unrelated items cluster together, for eg. food essentials, bath essentials, bath equipment, rehab equipment
G = to_graph(cat_list)
final = connected_components(G)
for cat in final:
  print(cat)

{'home decor'}
{'household items'}
{'apparel', 'apparels test', 'apparels'}
{'rehab epquipment', 'earring set', 'polishing agent', 'small goods', 'chyawanprash range', 'medical equipment', 'bath essentials', 'power equipment', 'bathing essentials', 'shpl range', 'food (liquid)', 'polishing essentials', 'food ingredients', 'industrial equipment', 'essentialgoods', 'food', 'food product', 'watches', 'misting', 'polishing agnt', 'medical essentials & equipments', 'earring', 'gardening', 'ring', 'food (solid)', 'essential epquipment', 'food products'}
{'small packages', 'medium sized packages', 'packages'}
{'travel gear', 'travel gears'}
{'personal health care', 'beauty & personal care', 'personalcare', 'health & personal care', 'beauty-beauty', 'beauty & personal', 'helth & personal care', 'personal care', 'beauty', 'beauty & persomal care'}
{'small home appliances'}
{'mobile accessories', 'mobile', 'accessories', 'tops', 'laptop & computer accessories'}
{'power bank'}
{'danglers'}
{'fake

In [ ]:
#cats = input_data['items.category'].unique()
#cats.sort()
#cats